In [7]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00


In [11]:
import torch
import pandas as pd
from google.colab import drive
from transformers import AutoModelForImageClassification, ViTHybridForImageClassification
from datasets import load_dataset

In [12]:
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
def get_model_param_info(model_names, output_file):
    model_data = []

    for model_name in model_names:
        print(f"\n🔍 Model: {model_name}")

        # Load model based on type
        if "hybrid" in model_name:
            model = ViTHybridForImageClassification.from_pretrained(model_name, num_labels=8, ignore_mismatched_sizes=True)
        else:
            model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=8, ignore_mismatched_sizes=True)

        # Identify trainable parameters
        for name, param in model.named_parameters():
            param.requires_grad = name.startswith(('classifier', 'distillation_classifier'))

        # Count parameters
        num_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"Total Parameters: {num_params:,} | Trainable: {trainable_params:,}")

        # Append to list
        model_data.append({
            "Model": model_name,
            "Total Parameters": num_params,
            "Trainable Parameters": trainable_params,
        })

    # Convert to DataFrame and save to Excel
    df_models = pd.DataFrame(model_data)
    return df_models

def get_dataset_info(dataset_path):
    dataset = load_dataset(dataset_path)

    # Get label names from the dataset metadata
    label_names = dataset["train"].features["label"].names  # Assuming all splits have the same labels

    class_data = []

    for split in dataset.keys():
        df = dataset[split].to_pandas()
        class_counts = df["label"].value_counts().reset_index()
        class_counts.columns = ["Class Index", "Count"]
        class_counts["Class"] = class_counts["Class Index"].map(lambda x: label_names[x])  # Map index to label
        class_counts["Split"] = split
        class_counts = class_counts[["Class", "Class Index", "Count", "Split"]]  # Reorder columns
        class_data.append(class_counts)

    # Merge all splits into one DataFrame
    df_dataset = pd.concat(class_data, ignore_index=True)
    return df_dataset

In [14]:
model_list = [
    "microsoft/resnet-152",
    "facebook/convnextv2-base-1k-224",
    "google/vit-hybrid-base-bit-384",
    "google/vit-base-patch16-224",
    "microsoft/swin-base-patch4-window7-224",
    "facebook/deit-base-patch16-224",
    "facebook/dinov2-base",
    "apple/mobilevit-small",
    "google/efficientnet-b0",
    "facebook/convnextv2-femto-1k-224",
    "WinKawaks/vit-tiny-patch16-224",
    "microsoft/swin-tiny-patch4-window7-224",
    "facebook/deit-tiny-patch16-224"
]
dataset_path = 'cvmil/rice-leaf-disease-augmented-v4'
output_file = "/content/drive/Shareddrives/CS198-Drones/[v4] FFT Module Evaluation/model_and_dataset_info.xlsx"

# Get model and dataset info
df_models = get_model_param_info(model_list, output_file)
df_dataset = get_dataset_info(dataset_path)

# Save both tables to the same Excel file
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    df_models.to_excel(writer, sheet_name="Model Parameters", index=False)
    df_dataset.to_excel(writer, sheet_name="Dataset Info", index=False)

print(f"\n✅ Results saved to {output_file}")


🔍 Model: microsoft/resnet-152


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-152 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([8, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Parameters: 58,160,200 | Trainable: 16,392

🔍 Model: facebook/convnextv2-base-1k-224


...

test-00000-of-00001.parquet:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8192 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/298 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/298 [00:00<?, ? examples/s]


✅ Results saved to /content/drive/Shareddrives/CS198-Drones/[v4] FFT Module Evaluation/model_and_dataset_info.xlsx
